In [33]:
import os
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt

# loading image data...
def data_loader(folder_dir):
    # folder_dir = "./data/images/"
    image_dataset = []
    mask_dataset = []
    for images in os.listdir(folder_dir+'/images'):
        image = cv2.imread(folder_dir+'/images/'+images, 1)
        # image = cv2.imread(folder_dir+'/'+images, 1)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        # image = cv2.resize(image, (128, 128))
        image = Image.fromarray(image)
        image = np.array(image)
        image_dataset.append(image)
        mask = cv2.imread(folder_dir+'/masks/10066_mask.png', 1)
        mask = cv2.cvtColor(mask,cv2.COLOR_BGR2RGB)
        mask = Image.fromarray(mask)
        mask = np.array(mask)
        mask_dataset.append(mask)
    return image_dataset, mask_dataset

In [34]:


image_dataset, mask_dataset = data_loader("./data")
image_dataset = np.array(image_dataset)
mask_dataset = np.array(mask_dataset)
mask_dataset.shape
image_dataset.shape

(1670, 480, 640, 3)

In [ ]:
import random

image_number = random.randint(0, len(image_dataset))
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(image_dataset[image_number])
plt.subplot(122)
plt.imshow(mask_dataset[image_number])
plt.show()